**Librerias y configuracion**

In [5]:
import pandas as pd
import unicodedata
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
from graphviz import Digraph

**Funcion para limpiar texto**

In [6]:
def limpiar_texto(texto):
    # Pasar a minúsculas
    texto = texto.lower()
    # Eliminar tildes
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto)
                    if unicodedata.category(c) != 'Mn')
    # Eliminar signos de puntuación
    texto = re.sub(r'[^\w\s]', '', texto)
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

**Cargar y limpiar el dataset**

In [ ]:
# Cargar dataset
dataset = pd.read_csv("datos/data_set.csv", skipinitialspace=True)

# Limpiar preguntas
dataset['Pregunta_limpia'] = dataset['Pregunta'].apply(limpiar_texto)

# Separar preguntas limpias y respuestas
preguntas = dataset['Pregunta_limpia'].values
respuestas = dataset['Respuesta'].values

**Vectorizacion y entrenamiento**

In [ ]:
# Vectorizar
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preguntas)

# Entrenar modelo
modelo = MultinomialNB()
modelo.fit(X, respuestas)

**Guardar modelo**

In [ ]:
# Guardar modelo y vectorizer
joblib.dump(modelo, "modelo_entrenado.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

**Funcion de prediccion**

In [ ]:
# Función de respuesta
def obtener_respuesta(pregunta_usuario):
    pregunta_limpia = limpiar_texto(pregunta_usuario)
    pregunta_vectorizada = vectorizer.transform([pregunta_limpia])
# Funcion probas para implementar modelo de confianza    
    probas = modelo.predict_proba(pregunta_vectorizada)[0]
    indice_max = probas.argmax()
    confianza = probas[indice_max]

    if confianza < 0.3:
# Guarda las preguntas que no entiende en un log        
         with open("log_desconocidas.txt", "a") as f:
            f.write(pregunta_usuario + "\n")
    return "No entendi lo que me preguntaste, me lo podes repetir?"

    respuesta = modelo.classes_[indice_max]
    return respuesta

**Prueba interactiva en vivo**

In [ ]:
# Interacción en bucle
while True:
    pregunta_usuario = input("Podes preguntarme algo o podes escribir 'salir': ")
    if pregunta_usuario.lower() == "salir":
        break
    print(obtener_respuesta(pregunta_usuario))